In [1]:
import pandas as pd 
from matplotlib import pyplot as plt 
import numpy as np 
import joblib 
from IPython.display import display
from tqdm import tqdm

In [2]:
from utilities import make_model
class feat_test_mdoel(make_model):
    def __init__(self , name , clf , gamma ,x ,y , train_data_loc):
        self.features = x.columns.to_list() 
        self.train_data_loc = train_data_loc
        make_model.__init__(self , name , clf , gamma , x ,y)

In [3]:
labels = ['AGN' , 'STAR' , 'YSO' , 'HMXB' , 'LMXB' , 'ULX' , 'CV' , 'PULSAR']
scores = ['precision' , 'recall' , 'f1_score']
models = ['all_feat_v1' ,'no_color' , 'no_gal_coord' , 'no_IR' , 'no_mw' , 'no_optical_uv']

In [5]:
def score_mean(file):
    all_feat = {
        'AGN' : {
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        } ,
        'YSO' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'STAR' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'HMXB' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'LMXB' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        } , 
        'ULX' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'CV':{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'PULSAR' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        }
    }
    for l in labels:
        for i in range(5):
            model = joblib.load(f'models/{file}_{i}.pkl')
            all_feat[l]['precision'].append(model.result['class_scores']['precision_score'][l])
            all_feat[l]['recall'].append(model.result['class_scores']['recall_score'][l])
            all_feat[l]['f1_score'].append(model.result['class_scores']['f1_score'][l])
    all_mean = {}
    for l in labels:
        all_mean[l] = {}
        for s in scores:
            all_mean[l][s] = np.mean(all_feat[l][s])*100
    all_std = {}
    for l in labels:
        all_std[l] = {}
        for s in scores:
            all_std[l][s] = np.std(all_feat[l][s])*100
    mean_df = pd.DataFrame(all_mean)
    std_df = pd.DataFrame(all_std)
    return mean_df , std_df

In [14]:
score_mean('no_IR')[0]

,AGN,STAR,YSO,HMXB,LMXB,ULX,CV,PULSAR
precision,94.998560,94.243615,88.494756,91.636027,95.551644,70.816205,58.260263,31.295840
recall,96.910230,93.182796,94.255875,90.213904,80.839161,63.412322,50.602410,26.930693
f1_score,95.944764,93.710153,91.284480,90.919059,87.578851,66.901563,54.142184,28.939978


In [10]:
final = []
models = ['all_feat_v1' , 'no_IR' ,  'no_optical_uv' , 'no_mw' ]
for mod in tqdm(models):
    mean , std = score_mean(mod)
    for l in labels:
        mean[l] = [f'{m:.1f}$\pm${s:.2f}' for m,s in zip(mean[l] , std[l])]
    final.append(mean)


100%|██████████| 4/4 [00:08<00:00,  2.08s/it]


In [11]:
final[0]

,AGN,STAR,YSO,HMXB,LMXB,ULX,CV,PULSAR
precision,96.8$\pm$0.14,96.1$\pm$0.08,92.8$\pm$0.17,91.8$\pm$0.24,94.8$\pm$1.33,71.7$\pm$1.23,60.6$\pm$1.16,42.1$\pm$1.93
recall,97.6$\pm$0.15,95.7$\pm$0.06,95.4$\pm$0.25,90.7$\pm$0.66,80.7$\pm$0.84,71.2$\pm$1.10,54.5$\pm$1.50,45.3$\pm$1.92
f1_score,97.2$\pm$0.11,95.9$\pm$0.07,94.1$\pm$0.20,91.2$\pm$0.36,87.2$\pm$0.60,71.5$\pm$1.05,57.4$\pm$1.33,43.7$\pm$1.89


In [22]:
final_df = pd.concat(final , keys=['All features' , 'no-IR' , 'no-optical-UV' , 'no-MW'])
final_df

AGN           STAR            YSO  \
All features  precision  96.8$\pm$0.14  96.1$\pm$0.08  92.8$\pm$0.17   
              recall     97.6$\pm$0.15  95.7$\pm$0.06  95.4$\pm$0.25   
              f1_score   97.2$\pm$0.11  95.9$\pm$0.07  94.1$\pm$0.20   
no-IR         precision  95.0$\pm$0.13  94.2$\pm$0.13  88.5$\pm$0.09   
              recall     96.9$\pm$0.15  93.2$\pm$0.07  94.3$\pm$0.20   
              f1_score   95.9$\pm$0.09  93.7$\pm$0.07  91.3$\pm$0.14   
no-optical-UV precision  96.0$\pm$0.16  95.3$\pm$0.22  92.4$\pm$0.18   
              recall     96.5$\pm$0.26  94.6$\pm$0.11  94.9$\pm$0.15   
              f1_score   96.3$\pm$0.17  95.0$\pm$0.10  93.7$\pm$0.08   
no-MW         precision  91.1$\pm$0.12  89.2$\pm$0.18  82.9$\pm$0.09   
              recall     95.1$\pm$0.09  88.2$\pm$0.11  89.4$\pm$0.33   
              f1_score   93.0$\pm$0.07  88.7$\pm$0.14  86.0$\pm$0.15   

                                  HMXB           LMXB            ULX  \
All features  precision  91.8$\pm$0.24  94.8$\pm$1.33  71.7$\pm$1.23   
              recall     90.7$\pm$0.66  80.7$\pm$0.84  71.2$\pm$1.10   
              f1_score   91.2$\pm$0.36  87.2$\pm$0.60  71.5$\pm$1.05   
no-IR         precision  91.6$\pm$0.30  95.6$\pm$1.30  70.8$\pm$1.39   
              recall     90.2$\pm$0.16  80.8$\pm$0.34  63.4$\pm$1.01   
              f1_score   90.9$\pm$0.16  87.6$\pm$0.68  66.9$\pm$0.91   
no-optical-UV precision  91.4$\pm$0.40  94.5$\pm$1.14  65.5$\pm$0.82   
              recall     91.0$\pm$0.31  81.4$\pm$0.34  64.9$\pm$0.30   
              f1_score   91.2$\pm$0.27  87.5$\pm$0.45  65.2$\pm$0.53   
no-MW         precision  92.3$\pm$0.45  94.5$\pm$1.02  61.9$\pm$1.33   
              recall     90.1$\pm$0.37  81.7$\pm$0.52  43.6$\pm$1.34   
              f1_score   91.2$\pm$0.39  87.6$\pm$0.55  51.2$\pm$1.36   

                                    CV         PULSAR  
All features  precision  60.6$\pm$1.16  42.1$\pm$1.93  
              recall     54.5$\pm$1.50  45.3$\pm$1.92  
              f1_score   57.4$\pm$1.33  43.7$\pm$1.89  
no-IR         precision  58.3$\pm$0.84  31.3$\pm$1.28  
              recall     50.6$\pm$1.90  26.9$\pm$2.02  
              f1_score   54.1$\pm$1.20  28.9$\pm$1.70  
no-optical-UV precision  56.7$\pm$1.15  35.3$\pm$1.66  
              recall     53.0$\pm$0.76  40.0$\pm$1.73  
              f1_score   54.8$\pm$0.67  37.5$\pm$1.61  
no-MW         precision  56.1$\pm$2.75  28.2$\pm$1.86  
              recall     45.1$\pm$1.04  18.6$\pm$0.74  
              f1_score   50.0$\pm$1.69  22.4$\pm$0.87

In [37]:
precision , recall , f1_score = {} ,{} , {}
for m in ['All features' , 'no-IR' , 'no-optical-UV' , 'no-MW']:
    precision[m] = (final_df.loc[m].loc['precision'].to_frame().T)
    recall[m] = (final_df.loc[m].loc['recall'].to_frame().T)
    f1_score[m] = (final_df.loc[m].loc['f1_score'].to_frame().T)
precision = pd.concat(precision).swaplevel(0,1)
recall = pd.concat(recall).swaplevel(0,1)
f1_score  = pd.concat(f1_score).swaplevel(0,1)
score = pd.concat([precision , recall , f1_score])
score

AGN           STAR            YSO  \
precision All features   96.8$\pm$0.14  96.1$\pm$0.08  92.8$\pm$0.17   
          no-IR          95.0$\pm$0.13  94.2$\pm$0.13  88.5$\pm$0.09   
          no-optical-UV  96.0$\pm$0.16  95.3$\pm$0.22  92.4$\pm$0.18   
          no-MW          91.1$\pm$0.12  89.2$\pm$0.18  82.9$\pm$0.09   
recall    All features   97.6$\pm$0.15  95.7$\pm$0.06  95.4$\pm$0.25   
          no-IR          96.9$\pm$0.15  93.2$\pm$0.07  94.3$\pm$0.20   
          no-optical-UV  96.5$\pm$0.26  94.6$\pm$0.11  94.9$\pm$0.15   
          no-MW          95.1$\pm$0.09  88.2$\pm$0.11  89.4$\pm$0.33   
f1_score  All features   97.2$\pm$0.11  95.9$\pm$0.07  94.1$\pm$0.20   
          no-IR          95.9$\pm$0.09  93.7$\pm$0.07  91.3$\pm$0.14   
          no-optical-UV  96.3$\pm$0.17  95.0$\pm$0.10  93.7$\pm$0.08   
          no-MW          93.0$\pm$0.07  88.7$\pm$0.14  86.0$\pm$0.15   

                                  HMXB           LMXB            ULX  \
precision All features   91.8$\pm$0.24  94.8$\pm$1.33  71.7$\pm$1.23   
          no-IR          91.6$\pm$0.30  95.6$\pm$1.30  70.8$\pm$1.39   
          no-optical-UV  91.4$\pm$0.40  94.5$\pm$1.14  65.5$\pm$0.82   
          no-MW          92.3$\pm$0.45  94.5$\pm$1.02  61.9$\pm$1.33   
recall    All features   90.7$\pm$0.66  80.7$\pm$0.84  71.2$\pm$1.10   
          no-IR          90.2$\pm$0.16  80.8$\pm$0.34  63.4$\pm$1.01   
          no-optical-UV  91.0$\pm$0.31  81.4$\pm$0.34  64.9$\pm$0.30   
          no-MW          90.1$\pm$0.37  81.7$\pm$0.52  43.6$\pm$1.34   
f1_score  All features   91.2$\pm$0.36  87.2$\pm$0.60  71.5$\pm$1.05   
          no-IR          90.9$\pm$0.16  87.6$\pm$0.68  66.9$\pm$0.91   
          no-optical-UV  91.2$\pm$0.27  87.5$\pm$0.45  65.2$\pm$0.53   
          no-MW          91.2$\pm$0.39  87.6$\pm$0.55  51.2$\pm$1.36   

                                    CV         PULSAR  
precision All features   60.6$\pm$1.16  42.1$\pm$1.93  
          no-IR          58.3$\pm$0.84  31.3$\pm$1.28  
          no-optical-UV  56.7$\pm$1.15  35.3$\pm$1.66  
          no-MW          56.1$\pm$2.75  28.2$\pm$1.86  
recall    All features   54.5$\pm$1.50  45.3$\pm$1.92  
          no-IR          50.6$\pm$1.90  26.9$\pm$2.02  
          no-optical-UV  53.0$\pm$0.76  40.0$\pm$1.73  
          no-MW          45.1$\pm$1.04  18.6$\pm$0.74  
f1_score  All features   57.4$\pm$1.33  43.7$\pm$1.89  
          no-IR          54.1$\pm$1.20  28.9$\pm$1.70  
          no-optical-UV  54.8$\pm$0.67  37.5$\pm$1.61  
          no-MW          50.0$\pm$1.69  22.4$\pm$0.87

In [39]:
print(score.replace({'_' , ' '}).to_latex(escape=False , multicolumn=True , column_format = 'lccccccccl' , multirow=True))

\begin{tabular}{lccccccccl}
\toprule
         &       &            AGN &           STAR &            YSO &           HMXB &           LMXB &            ULX &             CV &         PULSAR \\
\midrule
\multirow{4}{*}{precision} & All features &  96.8$\pm$0.14 &  96.1$\pm$0.08 &  92.8$\pm$0.17 &  91.8$\pm$0.24 &  94.8$\pm$1.33 &  71.7$\pm$1.23 &  60.6$\pm$1.16 &  42.1$\pm$1.93 \\
         & no-IR &  95.0$\pm$0.13 &  94.2$\pm$0.13 &  88.5$\pm$0.09 &  91.6$\pm$0.30 &  95.6$\pm$1.30 &  70.8$\pm$1.39 &  58.3$\pm$0.84 &  31.3$\pm$1.28 \\
         & no-optical-UV &  96.0$\pm$0.16 &  95.3$\pm$0.22 &  92.4$\pm$0.18 &  91.4$\pm$0.40 &  94.5$\pm$1.14 &  65.5$\pm$0.82 &  56.7$\pm$1.15 &  35.3$\pm$1.66 \\
         & no-MW &  91.1$\pm$0.12 &  89.2$\pm$0.18 &  82.9$\pm$0.09 &  92.3$\pm$0.45 &  94.5$\pm$1.02 &  61.9$\pm$1.33 &  56.1$\pm$2.75 &  28.2$\pm$1.86 \\
\cline{1-10}
\multirow{4}{*}{recall} & All features &  97.6$\pm$0.15 &  95.7$\pm$0.06 &  95.4$\pm$0.25 &  90.7$\pm$0.66 &  80.7$\pm$0.84 &  7

In [20]:
final_df.loc['no-MW']

,AGN,STAR,YSO,HMXB,LMXB,ULX,CV,PULSAR
precision,91.1$\pm$0.12,89.2$\pm$0.18,82.9$\pm$0.09,92.3$\pm$0.45,94.5$\pm$1.02,61.9$\pm$1.33,56.1$\pm$2.75,28.2$\pm$1.86
recall,95.1$\pm$0.09,88.2$\pm$0.11,89.4$\pm$0.33,90.1$\pm$0.37,81.7$\pm$0.52,43.6$\pm$1.34,45.1$\pm$1.04,18.6$\pm$0.74
f1_score,93.0$\pm$0.07,88.7$\pm$0.14,86.0$\pm$0.15,91.2$\pm$0.39,87.6$\pm$0.55,51.2$\pm$1.36,50.0$\pm$1.69,22.4$\pm$0.87


In [66]:
print(final_df.replace({'_' , ' '}).to_latex(escape=False , multicolumn=True , column_format = 'lccccccccl'))

\begin{tabular}{lccccccccl}
\toprule
                            &          &            AGN &           STAR &            YSO &           HMXB &           LMXB &            ULX &             CV &         PULSAR \\
\midrule
All 41 features & precision &  96.8$\pm$0.14 &  96.1$\pm$0.08 &  92.8$\pm$0.17 &  91.8$\pm$0.24 &  94.8$\pm$1.33 &  71.7$\pm$1.23 &  60.6$\pm$1.16 &  42.1$\pm$1.93 \\
                            & recall &  97.6$\pm$0.15 &  95.7$\pm$0.06 &  95.4$\pm$0.25 &  90.7$\pm$0.66 &  80.7$\pm$0.84 &  71.2$\pm$1.10 &  54.5$\pm$1.50 &  45.3$\pm$1.92 \\
                            & f1_score &  97.2$\pm$0.11 &  95.9$\pm$0.07 &  94.1$\pm$0.20 &  91.2$\pm$0.36 &  87.2$\pm$0.60 &  71.5$\pm$1.05 &  57.4$\pm$1.33 &  43.7$\pm$1.89 \\
color features removed & precision &  96.6$\pm$0.12 &  96.0$\pm$0.17 &  91.8$\pm$0.26 &  91.9$\pm$0.47 &  95.0$\pm$0.61 &  71.6$\pm$0.90 &  61.8$\pm$1.38 &  40.4$\pm$1.35 \\
                            & recall &  97.6$\pm$0.10 &  95.2$\pm$0.14 &  95.3$\p

In [ ]:
mean = mean.reset_index()
display(mean)
print(mean.to_latex(escape=False))